# Face Detection using OpenCV Caffemodel

In [12]:
import numpy as np
import cv2

In [13]:
print("[INFO] DEFINING PARAMETERS...")
    
# Files needed for face detection
prototxt_path = './FaceDetection_OpenCV_DNN_Files/deploy.prototxt.txt'
weights_path = './FaceDetection_OpenCV_DNN_Files/res10_300x300_ssd_iter_140000.caffemodel'

# Minimum probability to filter weak detections
MIN_CONFIDENCE = 0.5

# ENTER THE PATH TO YOUR VIDEO HERE
path_to_video = './videos/video1.m4v'

[INFO] DEFINING PARAMETERS...


In [14]:
print("[INFO] LOADING CAFFE FACE DETECTION MODEL...")

# Load the face detector model
# This face detection model is a pretrained model and is supported by opencv
face_detection_dnn = cv2.dnn.readNet(prototxt_path, weights_path)

[INFO] LOADING CAFFE FACE DETECTION MODEL...


In [15]:
# Detect all faces in GIVEN image using the pretrained model

def detect_faces(image):    
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    # we resize to 300×300 pixels and perform mean subtraction.

    # BLOB stands for Binary Large OBject and refers to a group of connected pixels in a binary image. 
    # The term "Large" indicates that only objects of a certain size are of interest 
    # and that the other "small" binary objects are usually noise.
    
    # pass the blob through the network and obtain the face detections

    face_detection_dnn.setInput(blob)
    face_detections = face_detection_dnn.forward()

    # Now we have all the detected faces and we know the location of each detected face
    # We will consider a face to be 'detected' if its confidence is greater than our minimum confidence
    
    return face_detections

In [16]:
def perform_face_mask_detection(frame):
    height = frame.shape[0]
    width = frame.shape[1]
    
    face_detections = detect_faces(frame)
    
    locations = []    # stores the locations of the detected faces
    confidences = []
    
    # Iterate through all the face detections
    for i in range(face_detections.shape[2]):
        # Confidence or probability of currenct detection
        confidence = face_detections[0, 0, i, 2]

        # Detections with confidence higher than our threshold are only considered
        if confidence > MIN_CONFIDENCE:
            
            # Computing boundaries of the bounding box of the detected face
            detected_rectangle = face_detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = detected_rectangle.astype("int")

            # Ensuring that the position of the detected face are within the boundaries of the image
            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(endX, width-1)
            endY = min(endY, height-1)

            locations.append((startX, startY, endX, endY))
            confidences.append(confidence)
            
    return locations, confidences

In [17]:
# Start the webcam and detect face masks in each of the frames

print('[INFO] STARTING THE VIDEO...')

vid = cv2.VideoCapture(path_to_video)


# loop over the frames from the video stream
while vid.isOpened():
    # resize each frame to have a maximum width of 400 pixels
    ret, frame = vid.read()

    if ret == True:
        height = frame.shape[0]
        width = frame.shape[1]
                
        # detect faces in the frame and determine if they are wearing a face mask or not
        locations, confidences = perform_face_mask_detection(frame)

        for (location, confidence) in zip(locations, confidences):
            (startX, startY, endX, endY) = location

            # Displaying rectangle
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0,200,0), 2)
            
            # Display confidence
            label = '{:.2f}%'.format(confidence*100)
            cv2.putText(frame, label, (startX, endY+30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            
            cv2.putText(frame, 'Press Q to exit', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 3)

        # Displaying the output frame
        cv2.imshow('Face Detection', frame)

        # Break if 'q' is pressed
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
    else:
        break

# Destroy all frames and close the videostream
vid.release()
cv2.destroyAllWindows()

[INFO] STARTING THE VIDEO...
